In [3]:
from src.finetuner import FineTuner
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainingArguments
)
from datasets import load_dataset

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [6]:
#constant
max_length = 30
text_col = 'content'
label_col = 'postprocess_quintuplet'
preprocess_type = 'p00'
SAVE_PATH = f'../models/quintuplet_t5_MLM-PT'
PRETRAINED_MODEL = "../models/pt-indot5-MLM_PT"#"../models/pt-indot5-MLM_PT" #"Wikidepia/IndoT5-base" 
TOKENIZER_PATH = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quintuplet/quintuplet_postprocessed_1000.csv'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)

In [7]:
# raw_dataset = load_dataset('csv', data_files={
#     'train' : f'{DATA_PATH}_train.csv',
#     'test' : f'{DATA_PATH}_test.csv',
#     'val' : f'{DATA_PATH}_val.csv'
# })
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
#raw_dataset = raw_dataset.filter(lambda x: x['is_comparative']==False)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=0.1, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-e45b5c90cd924547/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 497.07it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ff1816f6c2c3ba58.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-64790f2f7fbaf197.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b27f5c338b779ada.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853b

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 930
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 104
    })
})

In [9]:
len_labels = []
len_inputs = []
for i in range(len(tokenized_dataset['train'])):
    len_labels.append(len(tokenized_dataset['train'][i]['labels']))
    len_inputs.append(len(tokenized_dataset['train'][i]['input_ids']))

In [10]:
sum(len_labels)/len(len_labels), sum(len_inputs)/len(len_inputs)

(106.0, 30.0)

In [11]:
tokenizer.decode(tokenized_dataset['test']['input_ids'][0])

'shope aja, di dom aku termasuk cepat dengan kondisi yang skrng ini</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [12]:
tokenizer.decode(tokenized_dataset['test']['labels'][0])

'(shopee,_,cepet dgn kondisi yg skrng ini,positive,delivery)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Model Training

In [13]:
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
#model.resize_token_embeddings(len(tokenizer))

In [14]:
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0003,
    #weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=10,
    save_total_limit=2,
    seed=42
)

In [15]:
finetuner = FineTuner(model=model, save_path=SAVE_PATH, tokenizer=tokenizer, 
                      train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['test'])

cuda:0


In [16]:
finetuner.fine_tune(training_args)

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1170 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
 10%|█         | 117/1170 [00:28<03:27,  5.08it/s]

{'eval_loss': 0.7088098526000977, 'eval_runtime': 0.5397, 'eval_samples_per_second': 192.715, 'eval_steps_per_second': 24.089, 'epoch': 1.0}


                                                    
 20%|██        | 234/1170 [01:20<03:07,  4.98it/s]

{'eval_loss': 0.336075097322464, 'eval_runtime': 0.5099, 'eval_samples_per_second': 203.975, 'eval_steps_per_second': 25.497, 'epoch': 2.0}


                                                    
 30%|███       | 351/1170 [02:10<02:40,  5.11it/s]

...

{'eval_loss': 0.29689183831214905, 'eval_runtime': 0.5136, 'eval_samples_per_second': 202.473, 'eval_steps_per_second': 25.309, 'epoch': 9.0}


                                                   
100%|██████████| 1170/1170 [08:10<00:00,  5.21it/s]

{'eval_loss': 0.30097663402557373, 'eval_runtime': 0.5049, 'eval_samples_per_second': 205.992, 'eval_steps_per_second': 25.749, 'epoch': 10.0}


100%|██████████| 1170/1170 [08:35<00:00,  2.27it/s]


{'train_runtime': 515.828, 'train_samples_per_second': 18.029, 'train_steps_per_second': 2.268, 'train_loss': 0.896937206463936, 'epoch': 10.0}


In [17]:
model.save_pretrained(SAVE_PATH)

# Training Pipeline

In [2]:
#constant
SAVE_PATH = f'../models/preprocess_T5'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_annottated_sample_dataset_clean.csv'
#variable
max_length = 128
text_col = 'content'
label_col = 'quadruplet'
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=20,
    save_total_limit=2,
)
#read dataset
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
#model training
for i in range(4):
    preprocess_type = f'p0{i}'
    print(f"[INFO] preprocessing dataset for experiment {preprocess_type}")
    #preprocess dataset
    preprocessor = Preprocessor(f'{preprocess_type}', tokenizer, max_length, text_col, label_col)
    tokenized_dataset = raw_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)
    splitted_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2, seed=42)
    print(f"[INFO] training model for experiment {preprocess_type}")
    model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
    finetuner = FineTuner(model=model, save_path=f"{SAVE_PATH}_{preprocess_type}", tokenizer=tokenizer, 
                      train_dataset=splitted_dataset['train'], eval_dataset=splitted_dataset['test'])
    finetuner.fine_tune(training_args)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-564c7144e159da05/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 420.36it/s]
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0619265f74d98b9e.arrow
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-5d08aae3a84c90c6.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4bae551a5fe5658f.arrow


[INFO] preprocessing dataset for experiment p0
[INFO] training model for experiment p0


c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda:0
cuda:0


  0%|          | 0/2260 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
  5%|▌         | 113/2260 [00:21<06:26,  5.56it/s]

...

{'eval_loss': 0.6941794157028198, 'eval_runtime': 1.3898, 'eval_samples_per_second': 162.617, 'eval_steps_per_second': 20.867, 'epoch': 19.0}


100%|██████████| 2260/2260 [14:38<00:00,  5.71it/s]

{'eval_loss': 0.6934356689453125, 'eval_runtime': 1.2494, 'eval_samples_per_second': 180.889, 'eval_steps_per_second': 23.211, 'epoch': 20.0}


100%|██████████| 2260/2260 [15:05<00:00,  2.49it/s]


{'train_runtime': 905.9792, 'train_samples_per_second': 19.934, 'train_steps_per_second': 2.495, 'train_loss': 1.0219785707186808, 'epoch': 20.0}


# Inference

In [18]:
from src.inference import ModelInference
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
)
from datasets import load_dataset

In [2]:
#constant
max_length = 20
text_col = 'content'
label_col = 'postprocess_quad'
preprocess_type = 'p00'
SAVE_PATH = f'../models/quintuplet_t5_MLM-PT'
PRETRAINED_MODEL = "../models/pt-indot5-MLM_PT" #"Wikidepia/IndoT5-base" "../models/pt-indot5-MLM_PT"
TOKENIZER_PATH = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_1k_clean.csv'
inference_length = 128

In [3]:
model = T5ForConditionalGeneration.from_pretrained(SAVE_PATH)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [4]:
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)

In [5]:
# raw_dataset = load_dataset('csv', data_files={
#     'train' : f'{DATA_PATH}_train.csv',
#     'test' : f'{DATA_PATH}_test.csv',
#     'val' : f'{DATA_PATH}_val.csv'
# })
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=100, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-afb3349ada6b2db0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 100.01it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-afb3349ada6b2db0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0909deac7a1dd8f9.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-afb3349ada6b2db0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-1f57e62b69639f7c.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-afb3349ada6b2db0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-27503055701ea386.arrow


In [18]:
tokenizer.decode(tokenized_dataset['test']['input_ids'][0])

'anjim shope coan po hari bahkan kebatal gk dikirim, padahal kata c</s>'

In [19]:
tokenizer.decode(tokenized_dataset['test']['labels'][0])

'(shopee,coan po,kebatal gk dikirim,negative,delivery)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [19]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['test'], model=model, 
                                 tokenizer=tokenizer, inference_len=512)
pred_text = model_inference.inference()

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:06<00:00,  1.86it/s]


In [54]:
pred_text

['(shopee,_,di dom aku termasuk cepet dengan kondisi yg sekarang ini,positive,delivery)',
 '(shopee,_,lemot banget,negative,website&apps)',
 '(tiktok,scorl,susah,negative,produk)',
 '(olx,_,mengaku budget nya minim,negative,price)',
 '(shopee,shopee express,aman kok,positive,delivery)',
 '(bukalapak,top up,uang gue habis,negative,website&apps)',
 '(shopee,baju,sold out terus,negative,produk)',
 '(shopee,tempe,lebih enak,positive,produk)',
 '(shopee,_,kepencet2 sendiri,negative,website&apps)',
 '(shopee,paket,belum ada kabar,negative,delivery)',
 '(shopee,cs chat,tidak masalah,negative,website&apps)',
 '(shopee,shopee express,lama banget,negative,delivery)',
 '(olx,beli hp ini di tokobagus,positive,produk)',
 '(shopee,_,jadi jamet gini,negative,produk)',
 '(shopee,sepatu,bagus gak,positive,produk)',
 '(shopee,paket,kiriman dari &amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;

In [20]:
test_dataset = splitted_dataset['test']
test_dataset = test_dataset.add_column(f'{preprocess_type}_model_prediction', pred_text)
test_dataset.to_csv('../Data/quintuplet/mlm-model_1000-data_predictions.csv')

Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3bb670568ff76367.arrow
Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 17.44ba/s]


61274

In [21]:
test_dataset

Dataset({
    features: ['aoriginal_id', 'content', 'final_sentiment', 'baseline_aspect_category', 'tweet_type', 'clean_tweet', 'label', 'corrected_label', 'keterangan', 'quintuplet_label', 'postprocess_quintuplet', 'p00_model_prediction'],
    num_rows: 104
})

using train data

In [27]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['train'], model=model, 
                                 tokenizer=tokenizer, inference_len=512)
pred_text = model_inference.inference()

100%|██████████| 117/117 [01:01<00:00,  1.90it/s]


In [28]:
train_dataset = splitted_dataset['train']
train_dataset = train_dataset.add_column(f'{preprocess_type}_model_prediction', pred_text)
train_dataset.to_csv('../Data/quintuplet/mlm-model_1000-data_predictions_train.csv')

Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-a9483cc938a1ab79.arrow
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 79.96ba/s]


543207

# Evaluate

In [22]:
import pandas as pd
from src.evaluator import Evaluator
from src.postprocessor import PostProcessor
from src.utils import extract_triplet

In [23]:
df = pd.read_csv('../Data/quintuplet/mlm-model_1000-data_predictions.csv')
df['postprocess_quintuplet'] = df['postprocess_quintuplet'].apply(lambda x:x.lower())
df.head(3)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,corrected_label,keterangan,quintuplet_label,postprocess_quintuplet,p00_model_prediction
0,1646959047613227008,"@discountfess Shopee aja, di dom aku termasuk ...",neutral,produk;,SUBJECTIVE,"Shopee aja, di dom aku termasuk cepet dgn kon...","(shopee, shopee, cepet dgn kondisi yg skrng in...","(shopee, _, cepet dgn kondisi yg skrng ini, po...",NaN,"(shopee, _, cepet dgn kondisi yg skrng ini, po...","(shopee,_,cepet dgn kondisi yg skrng ini,posit...","(shopee,_,masuk akal dengan kondisi sekarang,p..."
1,1647278516495613954,shopee jelek bgt dah lemotnyaaaa,negative,produk;,SUBJECTIVE,shopee jelek bgt dah lemotnyaaaa,"(shopee, shopee, jelek bgt dah lemotnyaaa, neg...","(shopee, _, jelek bgt dah lemotnyaaa, negative...",NaN,"(shopee, _, jelek bgt dah lemotnyaaa, negative...","(shopee,_,jelek bgt dah lemotnyaaa,negative,we...","(shopee,_,lemot banget,negative,website&apps)"
2,1646549004757639168,@itsbeautyfess Aku juga susah nyari warna ston...,negative,produk;,SUBJECTIVE,Aku juga susah nyari warna stone pusing bgt s...,"(shopee, warna stone, susah nyari, negative, p...","(shopee, warna stone, susah nyari, negative, p...",NaN,"(shopee, warna stone, susah nyari, negative, p...","(shopee,warna stone,susah nyari,negative,produ...","(shopee,warna stone,pusing bgt scorl,negative,..."


In [24]:
postprocessor = PostProcessor(use_postprocess=False)
evaluator = Evaluator(task_type='quintuplet', postprocessor=postprocessor)

In [25]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['postprocess_quintuplet'])

100%|██████████| 104/104 [00:00<00:00, 52009.97it/s]


In [26]:
raw_scores

{'precision': 0.04672897196261682,
 'recall': 0.04424778761061947,
 'f1': 0.04545454545454546}

train score

In [29]:
df = pd.read_csv('../Data/quintuplet/mlm-model_1000-data_predictions_train.csv')
df['postprocess_quintuplet'] = df['postprocess_quintuplet'].apply(lambda x:x.lower())
df.head(3)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,corrected_label,keterangan,quintuplet_label,postprocess_quintuplet,p00_model_prediction
0,1647261067511496704,eh mau lebaran kyk gini ekspedisi kan lagi hec...,negative,produk;,SUBJECTIVE,eh mau lebaran kyk gini ekspedisi kan lagi hec...,"(shopee, shopee xpress, ekpsedisi kan lagi hec...",NaN,NaN,"(shopee, shopee xpress, ekpsedisi kan lagi hec...","(shopee,shopee xpress,ekpsedisi kan lagi hecti...","(shopee,shopee xpress,cepet pake,positive,deli..."
1,1646546316204597248,@tanyakanrl asli pernah lewat beranda tbtb ada...,negative,delivery; website&apps; produk;,SUBJECTIVE,asli pernah lewat beranda tbtb ada yg jual ko...,"(shopee, konten dewasa, parah si kok bisa lolo...",NaN,NaN,"(shopee, konten dewasa, parah si kok bisa lolo...","(shopee,konten dewasa,parah si kok bisa lolos ...","(shopee,konten dewasa,parah si kok bisa lolos ..."
2,1647243563443388416,"Adeuh, ini barang gua dri shopee blm nyampai2 ...",negative,delivery; customerservice; produk;,SUBJECTIVE,"Adeuh, ini barang gua dri shopee blm nyampai2 ...","(shopee, barang, blm nyampai2 jg woy, negative...",NaN,NaN,"(shopee, barang, blm nyampai2 jg woy, negative...","(shopee,barang,blm nyampai2 jg woy,negative,de...","(shopee,barang,blm nyampai2 jg woy,negative,de..."


In [30]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['postprocess_quintuplet'])

100%|██████████| 930/930 [00:00<00:00, 116092.34it/s]


In [31]:
raw_scores

{'precision': 0.6859852476290832,
 'recall': 0.6314258001939864,
 'f1': 0.6575757575757576}